In [ ]:
import os
import sys
from pathlib import Path
import numpy as np
repo_root = Path.cwd().parent
sys.path.append(str(repo_root))

folder_name = os.path.join(r'\\10.229.121.108','Workspace','PAT LINEAR','PATDOTUS','01')
from src.load_data_utils import list_subfolders, linear_us_param, linear_pa_param
import matplotlib.pyplot as plt

all_folders = list_subfolders(folder_name)
print(f"Found {len(all_folders)} image folders")
info_US = linear_us_param()
info_PA = linear_pa_param()

In [ ]:
# Estimate background mu at PAT wavelengths
from src.fluence_utils import estimate_so2_from_dot, query_bkg_mua_for_pa, fit_bkg_mus_for_pa
from src.recon_utils import generate_mu_init

lambda_list_dot = [730   , 785   , 808   , 830]
mua_list_dot    = [0.032 , 0.038 , 0.037 , 0.041]
mus_list_dot    = [7.860 , 7.489 , 7.155 , 7.830]
decomp_dot      = estimate_so2_from_dot(lambda_list_dot , mua_list_dot , verbose=True)
lambda_list_pat = [770   , 780   , 800   , 830]
bkg_mua_pat = query_bkg_mua_for_pa(lambda_list_pat , decomp_dot['c_oxy'] , decomp_dot['c_deoxy'])
print("Background mua at PAT wavelengths are " , bkg_mua_pat['Background mua'] , " cm\u207B\u00B9")
bkg_mus_pat = fit_bkg_mus_for_pa(lambda_list_dot, mus_list_dot, lambda_list_pat)
print("Background mus at PAT wavelengths are " , bkg_mus_pat['Background mus'] , " cm\u207B\u00B9")
wavelength_index = 0
pat_folder_index = 24
mu_a_bkg  = bkg_mua_pat['Background mua'][wavelength_index]
mu_sp_bkg = bkg_mus_pat['Background mus'][wavelength_index]
bbox_cm = (-1.25, 1.25, 0.10, 3.00)
scaling_factor = (4,5)
dz = info_PA.c / info_PA.fc / 2.0 * scaling_factor[1] * 100.0
dx = info_PA.c / info_PA.fc / 2.0 * scaling_factor[0] * 100.0
pixel_cm = (dz , dx)
input_dir = all_folders[pat_folder_index]
res = generate_mu_init(input_dir , info_US , info_PA , 
                       mu_a_mean_cm=mu_a_bkg , mu_s_mean_cm = mu_sp_bkg,
                       bbox_cm = bbox_cm , pixel_size_cm = pixel_cm)

In [ ]:
plt.figure(figsize=(4,3),dpi=400)
im = plt.imshow(res["US image"],cmap='gray',extent=[-info_US.FOV*50 , info_US.FOV*50 , 0.0 , 4.0], vmax = 0.73)
plt.axis('off')
cbar = plt.colorbar(im, fraction=0.046, pad=0.04)
cbar.set_label(r"US", fontsize=12)
plt.show()

In [ ]:
results_folder = 'notebooks'
mus_770 = np.load(os.path.join(repo_root, results_folder,'mus-770.npy'))
mus_780 = np.load(os.path.join(repo_root, results_folder,'mus-780.npy'))
mus_800 = np.load(os.path.join(repo_root, results_folder,'mus-800.npy'))
mus_830 = np.load(os.path.join(repo_root, results_folder,'mus-830.npy'))
mus_avg = 0.25*(mus_770 + mus_780 + mus_800 + mus_830)
bbox_cm = (-1.25,1.25,0.1,3.0)
xmin , xmax , zmin , zmax = bbox_cm
plt.figure(figsize=(4,3),dpi=400)
im = plt.imshow(mus_avg[1:-1,1:-1],cmap='turbo',extent=[xmin , xmax , zmin , zmax])
plt.axis('off')
cbar = plt.colorbar(im, fraction=0.046, pad=0.04)
cbar.set_label(r"$\mu_s^{'}$ (cm$^{-1}$)", fontsize=12)
plt.show()

In [ ]:
mua_770 = np.load(os.path.join(repo_root, results_folder,'mua-770.npy'))
mua_780 = np.load(os.path.join(repo_root, results_folder,'mua-780.npy'))
mua_800 = np.load(os.path.join(repo_root, results_folder,'mua-800.npy'))
mua_830 = np.load(os.path.join(repo_root, results_folder,'mua-830.npy'))
mua_all = np.stack((mua_770 , mua_780 , mua_800 , mua_830) , axis=-1)
print(mua_all.shape)

In [ ]:
from scipy.optimize import nnls
from tqdm import tqdm
wavelength_list = [770,780,800,830]
Nz , Nx , N_lambda = mua_all.shape
e_hbo = [650.0  , 710.0  , 816.0 , 974.0]
e_hbr = [1311.88, 1075.44, 761.72, 693.04]
e_hbo = np.asarray(e_hbo, dtype=np.float32).ravel()
e_hbr = np.asarray(e_hbr, dtype=np.float32).ravel()
e_all = np.stack([e_hbo,e_hbr],axis=1)*1e-3
c_hbo = np.zeros((Nz,Nx),dtype=np.float32)
c_hbr = np.zeros((Nz,Nx),dtype=np.float32)

for iz in tqdm(range(Nz),'Computing linear unmixing', leave=False):
    for ix in range(Nx):
        mua_vec = mua_all[iz,ix,:]
        c,_ = nnls(e_all, mua_vec)
        c_hbo[iz,ix] , c_hbr[iz,ix] = c

c_hbo /= 1e3
c_hbr /= 1e3
print('Linear unmixing complete')

In [ ]:
from matplotlib.colors import LinearSegmentedColormap
def get_oxy_colormap():
    colors = [
        (0.00, '#000000'),  # black: unphysiologically low
        (0.05, '#081D58'),  # dark navy blue
        (0.20, '#253494'),  # blue
        (0.35, '#41B6C4'),  # cyan-teal transition
        (0.50, '#1A9850'),  # green (physiologic)
        (0.65, '#F6E840'),  # yellow (~moderate-high)
        (0.80, '#FC8D59'),  # light orange-red
        (1.00, '#99000D')   # dark red (highly oxygenated)
    ]
    return LinearSegmentedColormap.from_list("oxy_cmap", colors)


In [ ]:
thb = c_hbo + c_hbr
with np.errstate(divide='ignore', invalid='ignore'):
    so2 = np.where(thb > 0, c_hbo / thb, 0.0)

In [ ]:
from src.recon_utils import enhance_vessels
import cv2
oxy_cmap = get_oxy_colormap()
so3 = cv2.bilateralFilter(src=so2,  d=0, sigmaColor = 0.05, sigmaSpace = 3)
plt.figure(figsize=(4,3),dpi=400)
im = plt.imshow(so3,cmap=oxy_cmap,extent=[xmin , xmax , zmin , zmax])
plt.axis('off')
cbar = plt.colorbar(im, fraction=0.046, pad=0.04)
cbar.set_label(r"$sO_2$", fontsize=12)
plt.show()

In [ ]:
thb_max = thb.max()
thb_norm = thb / thb_max
thb2 , vesselness = enhance_vessels(thb_norm,sigmas=(1, 2), vessel_weight=0.8)
plt.figure(figsize=(4,3),dpi=400)
im = plt.imshow(thb2*thb_max*0.5*1e6,cmap='hot',extent=[xmin , xmax , zmin , zmax])
plt.axis('off')
cbar = plt.colorbar(im, fraction=0.046, pad=0.04)
cbar.set_label(r"THb ($\mu$M)", fontsize=12)
plt.show()

In [ ]:
mua_isosbestic = mua_800
mua_isosbestic_max = mua_isosbestic.max()
mua_isosbestic_norm = mua_isosbestic / mua_isosbestic_max
mua_plot , vesselness = enhance_vessels(mua_isosbestic_norm,sigmas=(1, 2), vessel_weight=0.8)
plt.figure(figsize=(4,3),dpi=400)
im = plt.imshow(mua_plot*mua_isosbestic_max,cmap='hot',extent=[xmin , xmax , zmin , zmax], vmax = 0.205)
plt.axis('off')
cbar = plt.colorbar(im,ticks=[0.10, 0.15, 0.20], fraction=0.046, pad=0.04)
cbar.set_label(r"$\mu_a$ (cm$^{-1}$)", fontsize=12)
plt.show()

In [ ]:
US_image = res["US image"]
Nz, Nx = US_image.shape
x = np.linspace(-info_US.FOV*50, info_US.FOV*50, Nx)
z = np.linspace(0.0, 4.0, Nz)

# Find index bounds corresponding to desired crop region
ix_min = np.searchsorted(x, xmin, side='left')
ix_max = np.searchsorted(x, xmax, side='right')
iz_min = np.searchsorted(z, zmin, side='left')
iz_max = np.searchsorted(z, zmax, side='right')

# Crop image
US_crop = US_image[iz_min:iz_max, ix_min:ix_max]
plt.figure(figsize=(4,3),dpi=400)
im = plt.imshow(US_crop,cmap='gray',extent=[xmin , xmax , zmin , zmax])
plt.axis('off')
cbar = plt.colorbar(im, fraction=0.046, pad=0.04)
cbar.set_label(r"US", fontsize=12)
plt.show()

In [ ]:
def overlay_so2_on_us(
    us_img,
    so2_map,
    cmap,
    us_clip=(1, 99),          # percentile clipping for US dynamic range
    so2_range=(0.0, 1.0),     # sO2 value range for colormap
    alpha=0.5,                # global transparency for sO2 layer
    mask=None,                # optional boolean mask to show only vessels/valid sO2
    extent=None,              # (xmin, xmax, zmin, zmax) for imshow extent if you use physical coords
    show_colorbar=True,
):

    us = np.asarray(us_img, dtype=np.float32)
    sO2 = np.asarray(so2_map, dtype=np.float32)
    # --- ultrasound contrast normalization (robust percentiles) ---
    lo, hi = np.percentile(us, us_clip)
    if hi > lo:
        usn = np.clip((us - lo) / (hi - lo), 0, 1)
    else:
        usn = np.zeros_like(us)

    # --- sO2 normalization & colormapping ---
    vmin, vmax = so2_range
    sO2n = np.clip((sO2 - vmin) / max(1e-9, (vmax - vmin)), 0, 1)
    sO2_rgba = cmap(sO2n)  # returns RGBA in [0,1]

    # --- apply alpha and mask ---
    if mask is not None:
        mask = mask.astype(bool)
    else:
        mask = np.ones_like(sO2n, dtype=bool)

    # prepare alpha channel
    A = np.zeros_like(sO2n, dtype=np.float32)
    A[mask] = alpha
    sO2_rgba[..., 3] = A  # set per-pixel alpha

    # --- plot ---
    fig, ax = plt.subplots(figsize=(4, 3), dpi=400,constrained_layout=True)
    im_us = ax.imshow(usn, cmap='gray', extent=extent)
    im_so2 = ax.imshow(sO2_rgba, extent=extent)
    ax.axis('off')

    if show_colorbar:
        # create a separate mappable for the colorbar using same cmap & norm
        from matplotlib.cm import ScalarMappable
        from matplotlib.colors import Normalize
        sm = ScalarMappable(norm=Normalize(vmin=vmin, vmax=vmax), cmap=cmap)
        sm.set_array([])
        cbar = plt.colorbar(sm, ax=ax, fraction=0.046, pad=0.04, ticks=[vmin, 0.5*(vmin+vmax), vmax])
        cbar.set_label(r"$sO_2$", fontsize=11)

    plt.show()
    return fig, ax

In [ ]:
US_crop.shape

In [ ]:
Nz , Nx = mus_avg.shape
US_crop_resized = cv2.resize(US_crop, (Nx,Nz), interpolation=cv2.INTER_CUBIC)
US_crop_resized = cv2.bilateralFilter(src = np.asarray(US_crop_resized,dtype=np.float32),  d=0, sigmaColor = 0.25, sigmaSpace = 2)
so4 = cv2.bilateralFilter(src = so3,  d=0, sigmaColor = 0.2, sigmaSpace = 2)
oxy_cmap = get_oxy_colormap()
mask = thb > 0.4*thb.max()
mask_uint8 = mask.astype(np.uint8)
mask_clean = cv2.morphologyEx(mask_uint8, cv2.MORPH_OPEN, np.ones((3,3), np.uint8))
mask_clean = cv2.morphologyEx(mask_clean, cv2.MORPH_CLOSE, np.ones((5,5), np.uint8))
mask_clean = mask_clean.astype(bool)
_ = overlay_so2_on_us(
    US_crop_resized, so4, oxy_cmap,
    us_clip=(0,99.9),
    so2_range=(0.0,1.0),
    alpha=0.2,
    mask = mask_clean,  # or a vessel mask / THb>threshold
    extent=[xmin,xmax,zmax,zmin],  # optional: x_min, x_max, z_max, z_min for correct axis orientation
)

In [ ]:
from matplotlib.cm import ScalarMappable
from matplotlib.colors import Normalize
def get_modified_hot(end_fraction=0.9):
    base_cmap = plt.get_cmap('hot')
    colors = base_cmap(np.linspace(0, end_fraction, 256))
    return LinearSegmentedColormap.from_list('hot_truncated', colors)

def overlay_thb_on_us(
    us_img,
    thb_map,
    cmap='inferno',          # good for intensity maps; try 'magma' or 'viridis' too
    us_clip=(1, 99),         # percentile stretch for US
    thb_clip=(5, 99),        # percentile stretch for THb → controls color + alpha
    thb_range=None,          # (min,max) in physical units; overrides thb_clip if given
    alpha_range=(0.0, 0.8),  # min/max alpha
    alpha_gamma=1.0,         # power-law on THb for alpha shaping (e.g., 0.7 boosts low)
    extent=None,             # imshow extent if using physical coords
    show_colorbar=True,
):

    # --- to arrays ---
    us = np.asarray(us_img, dtype=np.float32)
    thb = np.asarray(thb_map, dtype=np.float32)

    # --- size match: resize THb to US shape ---
    if thb.shape != us.shape:
        thb = cv2.resize(thb, (us.shape[1], us.shape[0]), interpolation=cv2.INTER_LINEAR)

    # --- US contrast normalization (robust percentiles) ---
    lo_us, hi_us = np.percentile(us, us_clip)
    usn = np.clip((us - lo_us) / max(1e-9, (hi_us - lo_us)), 0, 1)

    # --- THb normalization (for color + alpha) ---
    if thb_range is not None:
        tmin, tmax = map(float, thb_range)
    else:
        tmin, tmax = np.percentile(thb, thb_clip)

    if tmax <= tmin:
        thb_norm = np.zeros_like(thb, dtype=np.float32)
    else:
        thb_norm = np.clip((thb - tmin) / (tmax - tmin), 0, 1).astype(np.float32)

    # --- colorize THb ---
    cmap_obj = plt.get_cmap(cmap) if isinstance(cmap, str) else cmap
    thb_rgba = cmap_obj(thb_norm)  # (H,W,4) in [0,1]

    # --- alpha from THb amplitude (with gamma + range) ---
    a_min, a_max = map(float, alpha_range)
    alpha = a_min + (a_max - a_min) * (thb_norm ** float(alpha_gamma))
    thb_rgba[..., 3] = alpha

    # --- plot ---
    fig, ax = plt.subplots(figsize=(4, 3),dpi=400,constrained_layout=True)
    ax.imshow(usn, cmap='gray',extent=extent)
    ax.imshow(thb_rgba, extent=extent)
    ax.axis('off')

    # --- colorbar showing THb scale (in normalized or physical units) ---
    if show_colorbar:
        # If user supplied physical thb_range, label in those units; else normalized to [tmin,tmax]
        norm = Normalize(vmin=tmin, vmax=tmax)
        sm = ScalarMappable(norm=norm, cmap=cmap_obj)
        sm.set_array([])
        cbar = plt.colorbar(sm, ax=ax, fraction=0.046, pad=0.04)
        cbar.set_label(r"THb ($\mu$M)", fontsize=12)

    plt.show()
    return fig, ax

In [ ]:
modified_hot = get_modified_hot(0.96)
_ = overlay_thb_on_us(
    US_crop_resized, thb2*thb_max*0.5e6,
    cmap=modified_hot,
    us_clip=(1,99),
    thb_clip=(3,99),
    alpha_range=(0.0, 0.7),
    alpha_gamma=0.8,
    extent=[xmin,xmax,zmax,zmin],
)

In [ ]:
def overlay_mua_on_us(
    us_img,
    thb_map,
    cmap='inferno',          # good for intensity maps; try 'magma' or 'viridis' too
    us_clip=(1, 99),         # percentile stretch for US
    thb_clip=(5, 99),        # percentile stretch for THb → controls color + alpha
    thb_range=None,          # (min,max) in physical units; overrides thb_clip if given
    alpha_range=(0.0, 0.8),  # min/max alpha
    alpha_gamma=1.0,         # power-law on THb for alpha shaping (e.g., 0.7 boosts low)
    extent=None,             # imshow extent if using physical coords
    cbar_ticks=(0.1, 0.15, 0.2),
    show_colorbar=True,
):

    # --- to arrays ---
    us = np.asarray(us_img, dtype=np.float32)
    thb = np.asarray(thb_map, dtype=np.float32)

    # --- size match: resize THb to US shape ---
    if thb.shape != us.shape:
        thb = cv2.resize(thb, (us.shape[1], us.shape[0]), interpolation=cv2.INTER_LINEAR)

    # --- US contrast normalization (robust percentiles) ---
    lo_us, hi_us = np.percentile(us, us_clip)
    usn = np.clip((us - lo_us) / max(1e-9, (hi_us - lo_us)), 0, 1)

    # --- THb normalization (for color + alpha) ---
    if thb_range is not None:
        tmin, tmax = map(float, thb_range)
    else:
        tmin, tmax = np.percentile(thb, thb_clip)

    if tmax <= tmin:
        thb_norm = np.zeros_like(thb, dtype=np.float32)
    else:
        thb_norm = np.clip((thb - tmin) / (tmax - tmin), 0, 1).astype(np.float32)

    # --- colorize THb ---
    cmap_obj = plt.get_cmap(cmap) if isinstance(cmap, str) else cmap
    thb_rgba = cmap_obj(thb_norm)  # (H,W,4) in [0,1]

    # --- alpha from THb amplitude (with gamma + range) ---
    a_min, a_max = map(float, alpha_range)
    alpha = a_min + (a_max - a_min) * (thb_norm ** float(alpha_gamma))
    thb_rgba[..., 3] = alpha

    # --- plot ---
    fig, ax = plt.subplots(figsize=(4, 3),dpi=400,constrained_layout=True)
    ax.imshow(usn, cmap='gray',extent=extent)
    ax.imshow(thb_rgba,extent=extent)
    ax.axis('off')

    if show_colorbar:
        # If user supplied physical thb_range, label in those units; else normalized to [tmin,tmax]
        norm = Normalize(vmin=tmin, vmax=tmax)
        sm = ScalarMappable(norm=norm, cmap=cmap_obj)
        sm.set_array([])
        cbar = plt.colorbar(sm, ax=ax, fraction=0.046, pad=0.04, ticks=cbar_ticks)
        cbar.set_label(r"$\mu_a$ (cm$^{-1}$)", fontsize=12)
        cbar.ax.tick_params(labelsize=10)

    plt.show()
    return fig, ax

In [ ]:
modified_hot = get_modified_hot(0.96)
_ = overlay_mua_on_us(
    US_crop_resized, mua_plot*mua_isosbestic_max,
    cmap=modified_hot,
    us_clip=(1,99),
    thb_clip=(1,99),
    alpha_range=(0.0, 0.7),
    alpha_gamma=0.8,
    extent=[xmin,xmax,zmax,zmin],
)

In [ ]:
from src.recon_utils import generate_mu_init
wavelength_index = 3
pat_folder_index = 27
mu_a_bkg  = bkg_mua_pat['Background mua'][wavelength_index]
mu_sp_bkg = bkg_mus_pat['Background mus'][wavelength_index]
bbox_cm = (-1.25, 1.25, 0.10, 3.00)
scaling_factor = (4,5)
dz = info_PA.c / info_PA.fc / 2.0 * scaling_factor[1] * 100.0
dx = info_PA.c / info_PA.fc / 2.0 * scaling_factor[0] * 100.0
pixel_cm = (dz , dx)
input_dir = all_folders[pat_folder_index]
res = generate_mu_init(input_dir , info_US , info_PA , 
                       mu_a_mean_cm=mu_a_bkg , mu_s_mean_cm = mu_sp_bkg,
                       bbox_cm = bbox_cm , pixel_size_cm = pixel_cm)
print(res.keys())

In [ ]:
pa_770 = res["PAT image"]

In [ ]:
pa_780 = res["PAT image"]

In [ ]:
pa_800 = res["PAT image"]

In [ ]:
pa_830 = res["PAT image"]

In [ ]:
pa_all = np.stack((pa_770/24.0,pa_780/24.5,pa_800/25.5,pa_830/22.5),axis=-1)
print(pa_all.shape)

In [ ]:
# linear unmixing for pa data
wavelength_list = [770,780,800,830]
Nz , Nx , N_lambda = pa_all.shape
e_hbo = [650.0  , 710.0  , 816.0 , 974.0]
e_hbr = [1311.88, 1075.44, 761.72, 693.04]
e_hbo = np.asarray(e_hbo, dtype=np.float32).ravel()
e_hbr = np.asarray(e_hbr, dtype=np.float32).ravel()
e_all = np.stack([e_hbo,e_hbr],axis=1)
c_hbo = np.zeros((Nz,Nx),dtype=np.float32)
c_hbr = np.zeros((Nz,Nx),dtype=np.float32)

pa_nb = np.empty_like(pa_all, dtype=np.float32)
for k in range(N_lambda):
    # 3x3 box filter, reflect borders to avoid dimming edges
    pa_nb[:, :, k] = cv2.blur(pa_all[:, :, k].astype(np.float32),
                              ksize=(3, 3),
                              borderType=cv2.BORDER_REFLECT)
    
for iz in tqdm(range(Nz),'Computing linear unmixing', leave=False):
    for ix in range(Nx):
        pa_vec = pa_nb[iz,ix,:]
        c,_ = nnls(e_all, pa_vec)
        c_hbo[iz,ix] , c_hbr[iz,ix] = c
print('Linear unmixing complete')

In [ ]:
Nz, Nx, _ = pa_all.shape
x = np.linspace(-info_PA.FOV*50, info_PA.FOV*50, Nx)
z = np.linspace(0.0, 4.0, Nz)

# Find index bounds corresponding to desired crop region
ix_min = np.searchsorted(x, xmin, side='left')
ix_max = np.searchsorted(x, xmax, side='right')
iz_min = np.searchsorted(z, zmin, side='left')
iz_max = np.searchsorted(z, zmax, side='right')

# Crop image
c_hbo_crop = c_hbo[iz_min:iz_max, ix_min:ix_max]
c_hbr_crop = c_hbr[iz_min:iz_max, ix_min:ix_max]

In [ ]:
thb = c_hbo_crop + c_hbr_crop
with np.errstate(divide='ignore', invalid='ignore'):
    so2 = np.where(thb > 0, c_hbo_crop / thb, 0.0)

In [ ]:
oxy_cmap = get_oxy_colormap()
so3 = cv2.resize(so2,(67,62))
so3 = cv2.bilateralFilter(src=so3,  d=0, sigmaColor = 0.1, sigmaSpace = 3)
plt.figure(figsize=(4,3),dpi=400)
im = plt.imshow(so3,cmap=oxy_cmap,extent=[xmin , xmax , zmin , zmax])
plt.axis('off')
cbar = plt.colorbar(im, fraction=0.046, pad=0.04)
cbar.set_label(r"$sO_2$", fontsize=12)
plt.show()

In [ ]:
thb = cv2.resize(thb,(67,62))
thb_max = thb.max()
thb_norm = thb / thb_max
thb2 , vesselness = enhance_vessels(thb_norm,sigmas=(1, 2,3,4), vessel_weight=0.2)
plt.figure(figsize=(4,3),dpi=400)
im = plt.imshow(thb2*thb_max*1e6,cmap='hot',extent=[xmin , xmax , zmin , zmax])
plt.axis('off')
cbar = plt.colorbar(im, fraction=0.046, pad=0.04)
cbar.set_label(r"THb (a.u.)", fontsize=12)
plt.show()

In [ ]:
mua_isosbestic = cv2.resize(pa_800,(67,62)) + 0.25*cv2.resize(pa_780,(67,62))
mua_isosbestic[0:6]=0.3*mua_isosbestic[0:6]
mua_isosbestic_max = mua_isosbestic.max()
mua_isosbestic_norm = mua_isosbestic / mua_isosbestic_max
#mua_plot , vesselness = enhance_vessels(mua_isosbestic_norm,sigmas=(1, 2, 3, 4), vessel_weight=0.1)
#mua_plot = cv2.bilateralFilter(np.asarray(mua_plot,dtype=np.float32) , d=0, sigmaColor = 0.05, sigmaSpace = 2)
plt.figure(figsize=(4,3),dpi=400)
im = plt.imshow(mua_isosbestic_norm,cmap='hot',extent=[xmin , xmax , zmin , zmax],vmax=0.9)
plt.axis('off')
cbar = plt.colorbar(im,fraction=0.046, pad=0.04)
cbar.set_label(r"PA", fontsize=12)
plt.show()

In [ ]:
roi_idx = (33,66,25,60)
idx_left, idx_right,idx_top,idx_bottom = roi_idx
so2_roi = so2[idx_top:idx_bottom,idx_left:idx_right]
thb_roi = thb[idx_top:idx_bottom,idx_left:idx_right]
mask = thb_roi > 0.55*thb.max()
so2_inside = so2_roi[mask]
thb_inside = thb_roi[mask]
print(so2_inside.mean())
print(thb_inside.mean()*0.5e6)

In [ ]:
mask = thb_roi > 0.55*thb.max()

plt.imshow(mask)
plt.colorbar()
plt.show()

In [ ]:
modified_hot = get_modified_hot(0.96)
_ = overlay_mua_on_us(
    US_crop_resized, mua_isosbestic_norm,
    cmap=modified_hot,
    us_clip=(1,99),
    thb_clip=(1,98),
    alpha_range=(0.0, 0.7),
    alpha_gamma=0.8,
    extent=[xmin,xmax,zmax,zmin],
)

In [ ]:
thb = cv2.resize(thb,(67,62))*1e6
thb3 = cv2.bilateralFilter(src=thb,  d=0, sigmaColor = 2, sigmaSpace = 3)

thb_max = thb3.max()
thb_norm = thb3 / thb_max
modified_hot = get_modified_hot(0.96)
_ = overlay_thb_on_us(
    US_crop_resized, thb_norm*thb_max*1e-30,
    cmap=modified_hot,
    us_clip=(1,99),
    thb_clip=(2,98),
    alpha_range=(0.0, 0.7),
    alpha_gamma=0.8,
    extent=[xmin,xmax,zmax,zmin],
)